In [27]:
# check if a code space protects against list of error channels
# Let C be a quantum code, and let P be the projector onto C.
# Suppose \Epsilon is quantum operation with operation elements {E_i}.
# A necessary and sufficient condition for the existence of
# an error-correction operation R correcting \Epsilon on C is that
# P E^dag_i E_j P = \alpha_{ij} P,
# for some Hermitian matrix \alpha of complex numbers.

In [28]:
import numpy as np


def are_matrices_scalar_multiple(mat1, mat2):
    # Step 1: Check if both matrices have the same shape
    if mat1.shape != mat2.shape:
        return False

    # Step 2: Flatten both matrices to 1D arrays
    flat_mat1 = mat1.flatten()
    flat_mat2 = mat2.flatten()

    # Step 3: Remove zero elements to avoid division by zero
    nonzero_indices = np.logical_and(flat_mat1 != 0, flat_mat2 != 0)
    flat_mat1 = flat_mat1[nonzero_indices]
    flat_mat2 = flat_mat2[nonzero_indices]

    # Check if either matrix is entirely zero
    if len(flat_mat1) == 0 or len(flat_mat2) == 0:
        return len(flat_mat1) == len(flat_mat2)

    # Step 4: Divide all elements of one array by the corresponding elements of the other array
    ratios = flat_mat1 / flat_mat2

    # Step 5: Check if the resulting array contains the same value for all elements
    return np.allclose(ratios, ratios[0])


# # Test the function
# mat1 = np.array([[2, 4], [6, 8]])
# mat2 = np.array([[1, 2], [3, 4]])

# print(are_matrices_scalar_multiple(mat1, mat2))  # Should return True

In [34]:
# define error channels as photon loss and phase flip
import numpy as np
from qutip import *
from qiskit.circuit.library import IGate, XGate

# use PEEP to check if bit repetition code protects against bit flips
logical0 = tensor(basis(2, 0), basis(2, 0), basis(2, 0))
logical1 = tensor(basis(2, 1), basis(2, 1), basis(2, 1))

Id = Qobj(IGate().to_matrix())
X = Qobj(XGate().to_matrix())

# define projector onto dual rail qubit
P = logical0 * logical0.dag() + logical1 * logical1.dag()

# define bit flip channel
E = []
p = 0.1
# \sqrt{(1-p)**3} I + \sqrt{p*(1-p)**2} X_1 + \sqrt{p*(1-p)**2} X_2 + \sqrt{p*(1-p)**2} X_3 + ...
E.append(np.sqrt((1 - p) ** 3) * tensor(Id, Id, Id))
E.append(tensor(np.sqrt(p * (1 - p) ** 2) * X, Id, Id))
E.append(tensor(Id, np.sqrt(p * (1 - p) ** 2) * X, Id))
E.append(tensor(Id, Id, np.sqrt(p * (1 - p) ** 2) * X))

# check if dual rail qubit protects against photon loss
for i in range(len(E)):
    for j in range(len(E)):
        print("PEEP for E" + str(i) + " and E" + str(j) + ":")
        # print((P * E[i].dag() * E[j] * P).tr() == P.tr() * P)
        left = (P * E[i].dag() * E[j] * P).full()
        right = P.full()
        print(are_matrices_scalar_multiple(left, right))

PEEP for E0 and E0:
True
PEEP for E0 and E1:
True
PEEP for E0 and E2:
True
PEEP for E0 and E3:
True
PEEP for E1 and E0:
True
PEEP for E1 and E1:
True
PEEP for E1 and E2:
True
PEEP for E1 and E3:
True
PEEP for E2 and E0:
True
PEEP for E2 and E1:
True
PEEP for E2 and E2:
True
PEEP for E2 and E3:
True
PEEP for E3 and E0:
True
PEEP for E3 and E1:
True
PEEP for E3 and E2:
True
PEEP for E3 and E3:
True


In [39]:
# define error channels as photon loss and phase flip
import numpy as np
from qutip import *

E = []

# define photon loss channel
# one 2 qubits, assume only one can be lost at a time
# each photon loss channel has 2 Kraus operators
# [[1, 0], [0, sqrt(1 - p)]] and [[0, sqrt(p)], [0, 0]]
# where p is the probability of photon loss
# E.append(tensor(qeye(2), Qobj([[1, 0], [0, np.sqrt(1 - 0.1)]])))
# E.append(tensor(qeye(2), Qobj([[0, np.sqrt(0.1)], [0, 0]])))
# E.append(tensor(Qobj([[1, 0], [0, np.sqrt(1 - 0.1)]]), qeye(2)))
# E.append(tensor(Qobj([[0, np.sqrt(0.1)], [0, 0]]), qeye(2)))

gamma = 0.1
a = destroy(2)
E.append(np.sqrt((1 - gamma) ** 2) * tensor(Id, Id))
E.append(tensor(np.sqrt(p * (1 - gamma)) * a, Id))
E.append(tensor(Id, np.sqrt(p * (1 - gamma)) * a))


# use PEEP to check if dual rail qubit protects against photon loss]
# define dual rail qubit
logical0 = tensor(basis(2, 0), basis(2, 1))
logical1 = tensor(basis(2, 1), basis(2, 0))

# define projector onto dual rail qubit
P = logical0 * logical0.dag() + logical1 * logical1.dag()

# check if dual rail qubit protects against photon loss
for i in range(len(E)):
    for j in range(len(E)):
        print("PEEP for E" + str(i) + " and E" + str(j) + ":")
        # print((P * E[i].dag() * E[j] * P).tr() == P.tr() * P)
        left = (P * E[i].dag() * E[j] * P).full()
        right = P.full()
        print(are_matrices_scalar_multiple(left, right))

PEEP for E0 and E0:
True
PEEP for E0 and E1:
True
PEEP for E0 and E2:
True
PEEP for E1 and E0:
True
PEEP for E1 and E1:
True
PEEP for E1 and E2:
True
PEEP for E2 and E0:
True
PEEP for E2 and E1:
True
PEEP for E2 and E2:
True


In [37]:
# check if dual rail qubit protects against photon loss
i, j = 0, 0
P * E[i].dag() * E[j] * P

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.   0.   0.   0.  ]
 [0.   0.81 0.   0.  ]
 [0.   0.   0.81 0.  ]
 [0.   0.   0.   0.  ]]